In [ ]:
# # check if gpu and CUDA are working
# !nvidia-smi

In [ ]:
# import torch

# #check if pytorch able to use gpu
# torch.cuda.get_device_name()

In [ ]:
%%capture
!pip install tensorflow

In [ ]:
# pip install keras==2.12.0

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
# Mounting google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Setting toolkit folder as working directory

df = pd.read_excel("/content/drive/My Drive/Dataset siswa/Data Bersih.xlsx")

In [ ]:
df.head()

,JenisKelamin,JenisTinggal,AlatTransportasi,JenjangPendidikanAyah,PekerjaanAyah,PenghasilanAyah,JenjangPendidikanIbu,PekerjaanIbu,PenghasilanIbu,NilaiTugas,NilaiUlangan,NilaiUTS,NilaiRapor
0,0,0,0,0,0,0,0,0,0,81.000000,70.000000,70.000000,75
1,1,0,0,1,1,1,1,1,1,91.500000,91.500000,66.500000,81
2,1,0,0,1,1,1,1,1,1,81.500000,82.000000,71.000000,78
3,1,0,0,0,0,1,1,0,2,89.333333,94.333333,74.333333,81
4,1,0,0,0,0,1,1,0,2,74.000000,75.500000,72.000000,67


### Defining the variables and splitting the data

In [ ]:
# Defining the variables
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]

In [ ]:
X.head()

,JenisKelamin,JenisTinggal,AlatTransportasi,JenjangPendidikanAyah,PekerjaanAyah,PenghasilanAyah,JenjangPendidikanIbu,PekerjaanIbu,PenghasilanIbu,NilaiTugas,NilaiUlangan,NilaiUTS
0,0,0,0,0,0,0,0,0,0,81.000000,70.000000,70.000000
1,1,0,0,1,1,1,1,1,1,91.500000,91.500000,66.500000
2,1,0,0,1,1,1,1,1,1,81.500000,82.000000,71.000000
3,1,0,0,0,0,1,1,0,2,89.333333,94.333333,74.333333
4,1,0,0,0,0,1,1,0,2,74.000000,75.500000,72.000000


In [ ]:
Y.head()

0    75
1    81
2    78
3    81
4    67
Name: NilaiRapor, dtype: int64

### Feature-Wise Normalization

In [ ]:
# Calculate the mean and standard deviation of each feature
mean_values = np.mean(X, axis=0)
std_values = np.std(X, axis=0)

In [ ]:
# Feature-wise normalization
X_scaled = (X - mean_values) / std_values

In [ ]:
print(X_scaled)

      JenisKelamin  JenisTinggal  AlatTransportasi  JenjangPendidikanAyah  \
0        -0.896364     -0.374110         -0.548547              -0.858272   
1         1.115618     -0.374110         -0.548547              -0.357513   
2         1.115618     -0.374110         -0.548547              -0.357513   
3         1.115618     -0.374110         -0.548547              -0.858272   
4         1.115618     -0.374110         -0.548547              -0.858272   
...            ...           ...               ...                    ...   
1243     -0.896364      1.407911          0.212105               0.644005   
1244     -0.896364      1.407911          0.212105               0.644005   
1245     -0.896364      1.407911          0.212105               0.644005   
1246     -0.896364     -0.374110          0.972757               2.146282   
1247     -0.896364     -0.374110         -0.548547               0.644005   

      PekerjaanAyah  PenghasilanAyah  JenjangPendidikanIbu  PekerjaanIbu  \

In [ ]:
# Membagi data menjadi 80% pelatihan dan 20% pengujian
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

In [ ]:
# Definisikan jumlah fold yang akan digunakan
k_values = [2, 3, 4, 5, 10]

In [ ]:
from keras import backend as K

def r2_squared(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
def make_model():
  model = Sequential()
  model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='linear'))
  model.compile(optimizer=Adam(), loss='mse', metrics=['mae', r2_squared, 'mse'])

  return model

model = make_model();

In [ ]:
def crossvalidation(k):
  kf = KFold(n_splits=k, shuffle=True, random_state=42)

  # Initialize variables to store per epoch MAE, MSE, R2 scores
  all_epoch_maes = []
  all_epoch_mse = []
  all_epoch_r2 = []

  # Dictionary to store results for each fold
  fold_results = {}

  for fold, (train_index, val_index) in enumerate (kf.split(X_scaled), 1):
    X_train, X_val = X_scaled.iloc[train_index], X_scaled.iloc[val_index]
    Y_train, Y_val = Y.iloc[train_index], Y.iloc[val_index]

  # Training model dengan 500 epoch
  history = model.fit(X_train, Y_train, epochs=500, verbose=0)

  # Get the per epoch MAE, MSE and R2 score
  epoch_maes = history.history['mae']
  all_epoch_maes.append(epoch_maes)
  # epoch_mse = history.history['mse']
  # all_epoch_mse.append(epoch_mse)
  epoch_r2 = history.history['r2_squared']
  all_epoch_r2.append(epoch_r2)

  # # Compute the average of per epoch MAE, MSE, R2 scores for all folds
  avg_epoch_maes = np.mean(all_epoch_maes, axis=0)
  # avg_epoch_mse = np.mean(all_epoch_mse, axis=0)
  avg_epoch_r2 = np.mean(all_epoch_r2, axis=0)

  avg_epoch_maes_score = np.mean(all_epoch_maes)
  # avg_epoch_mse_score = np.mean(all_epoch_mse)
  avg_epoch_r2_score = np.mean(all_epoch_r2)

  # Find the epoch with the minimum validation MAE
  best_epoch = np.argmin(avg_epoch_maes)
  best_epoch_array.append(best_epoch)

  # Stored results for the fold
  fold_results[f'Fold (fold)'] = {
      'Best Epoch': best_epoch,
      'Validation MAE': epoch_maes[best_epoch],
      # 'Validation MSE': epoch_mse[best_epoch],
      'Validation R2': epoch_r2[best_epoch],
  }

  # Store evaluation metrics results
  fold_results['Averaged evaluation metrics result'] = {
      'Average MAE': avg_epoch_maes_score,
      # 'Average MSE': avg_epoch_mse_score,
      'Average R2' : avg_epoch_r2_score
  }

  # Print all results
  print("\nResults:")
  for key, value in fold_results.items():
    print(f"\n{key}:")
    for sub_key, sub_value in value.items():
      print(f"{sub_key}: {sub_value}")

In [ ]:
import traceback

In [ ]:
best_epoch_array = [];

In [ ]:
# Looping untuk setiap nilai k
for k in k_values:
  print(f"\nPerforming {k}-fold cross-validation")
  crossvalidation(k);


Performing 2-fold cross-validation

Results:

Fold (fold):
Best Epoch: 447
Validation MAE: 0.3358768820762634
Validation R2: 0.9950113296508789

Averaged evaluation metrics result:
Average MAE: 0.5732399787902832
Average R2: 0.9874744658470154

Performing 3-fold cross-validation

Results:

Fold (fold):
Best Epoch: 459
Validation MAE: 0.41147729754447937
Validation R2: 0.9930285215377808

Averaged evaluation metrics result:
Average MAE: 0.6268670719861984
Average R2: 0.9851437863111496

Performing 4-fold cross-validation

Results:

Fold (fold):
Best Epoch: 421
Validation MAE: 0.49735400080680847
Validation R2: 0.9877361059188843

Averaged evaluation metrics result:
Average MAE: 0.6951567497849465
Average R2: 0.9798885889053345

Performing 5-fold cross-validation

Results:

Fold (fold):
Best Epoch: 347
Validation MAE: 0.5063470005989075
Validation R2: 0.9873086214065552

Averaged evaluation metrics result:
Average MAE: 0.7004303447008133
Average R2: 0.9792024006843567

Performing 10-fol

In [ ]:
avg_best_epoch = int(np.mean(best_epoch_array))
print(f"\nrata-rata tiap epoch: {avg_best_epoch}")


rata-rata tiap epoch: 454


#BUILDING FINAL MODEL WITHOUT HYPERPARAMETER TUNING

In [ ]:
def build_final_model(best_epoch):
  try:
    # Build the final model
    # final_model = Sequential()
    # final_model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
    # final_model.add(Dense(64, activation='relu'))
    # final_model.add(Dense(1, activation='linear'))
    final_model = make_model()

    final_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', r2_squared])

    # early_stopping = EarlyStopping(monitor='r2_squared', patience=500, restore_best_weights=True)

    # Train the final model on the entire dataset
    # final_model.fit(X_train, Y_train, epochs=best_epoch, batch_size=32, verbose=1, callbacks=[early_stopping])
    final_model.fit(X_train, Y_train, epochs=best_epoch, batch_size=128, verbose=1)
    return final_model
  except Exception as e:
    # Print the exception message
    print(f"Exception: {e}")

    # Get the exception traceback
    traceback_info = traceback.format_exc()

    # Print the traceback information
    print(traceback_info)

In [ ]:
print(f"\nBuilding final model with epoch {avg_best_epoch}")
final_model = build_final_model(avg_best_epoch)


Building final model with epoch 454
Epoch 1/454
8/8 [==============================] - 1s 3ms/step - loss: 5269.6133 - mae: 72.1186 - r2_squared: -79.3513
Epoch 2/454
8/8 [==============================] - 0s 3ms/step - loss: 5176.6123 - mae: 71.4690 - r2_squared: -80.7811
Epoch 3/454
8/8 [==============================] - 0s 3ms/step - loss: 5077.3745 - mae: 70.7679 - r2_squared: -78.8695
Epoch 4/454
8/8 [==============================] - 0s 3ms/step - loss: 4959.6035 - mae: 69.9300 - r2_squared: -77.7878
Epoch 5/454
8/8 [==============================] - 0s 3ms/step - loss: 4817.3135 - mae: 68.8961 - r2_squared: -73.5261
Epoch 6/454
8/8 [==============================] - 0s 3ms/step - loss: 4641.2637 - mae: 67.5980 - r2_squared: -69.3271
Epoch 7/454
8/8 [==============================] - 0s 3ms/step - loss: 4427.3135 - mae: 65.9870 - r2_squared: -66.0666
Epoch 8/454
8/8 [==============================] - 0s 5ms/step - loss: 4172.3311 - mae: 64.0091 - r2_squared: -64.3900
Epoch 9/454

In [ ]:
# Evaluate the final model
predictions = final_model.predict(X_test)

8/8 [==============================] - 0s 2ms/step


In [ ]:
final_mse = mean_squared_error(Y_test, predictions)
print(f"Final Mean Squared Error on the best model: {final_mse:.2f}")

Final Mean Squared Error on the best model: 34.30


In [ ]:
final_mae = mean_absolute_error(Y_test, predictions)
print(f"Final Mean Absolute Error on the best model: {final_mae:.2f}")

Final Mean Absolute Error on the best model: 4.48


In [ ]:
final_r2 = r2_score(Y_test, predictions)
print(f"Final r2_score on the best model: {final_r2:.2f}")

Final r2_score on the best model: 0.39


In [ ]:
print(final_model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5057 (19.75 KB)
Trainable params: 5057 (19.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


# BUILDING MODEL WITH HYPERPARAMETER TUNING

In [ ]:
%%capture
!pip install keras==2.12.0
!pip install --upgrade tensorflow joblib
!pip install scikeras

In [ ]:
from sklearn.model_selection import GridSearchCV
# from keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasRegressor

In [ ]:
param_grid = {
    # 'lr': [0.001, 0.01, 0.1],
    'optimizer': ['adam', 'sgd'],
    # 'batch_size': [32, 64, 128, 256, 512]
    'batch_size': [128, 256],
},

In [ ]:
# Create a KerasRegressor wrapper for use with GridSearchCV
keras_regressor = KerasRegressor(model=make_model)

In [ ]:
grid_search = GridSearchCV(estimator=keras_regressor, param_grid=param_grid, cv=2, scoring='r2', n_jobs=10)

In [ ]:
print(keras_regressor.get_params().keys())

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs'])


In [ ]:
grid_result = grid_search.fit(X_train, Y_train)

8/8 [==============================] - 1s 3ms/step - loss: 5208.2031 - mae: 71.6891 - r2_squared: -80.8196 - mse: 5208.2031


In [ ]:
# Get the best parameters
best_optimizer = grid_result.best_params_['optimizer']
best_batch_size = grid_result.best_params_['batch_size']

# Display the best parameters
print(f"Best Optimizer: {best_optimizer}")
print(f"Best Batch Size: {best_batch_size}")

Best Optimizer: adam
Best Batch Size: 128


In [ ]:
def build_tuned_final_model(optimizer, batch_size, best_epoch):
  tuned_final_model = make_model()
  tuned_final_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', r2_squared])
  tuned_final_model.fit(X_train, Y_train, epochs=best_epoch, batch_size=128, verbose=1)
  return tuned_final_model

In [ ]:
tuned_final_model = build_tuned_final_model(best_optimizer, best_batch_size, avg_best_epoch)

Epoch 1/454
8/8 [==============================] - 1s 3ms/step - loss: 5289.5312 - mae: 72.2628 - r2_squared: -81.5018
Epoch 2/454
8/8 [==============================] - 0s 3ms/step - loss: 5194.3877 - mae: 71.5988 - r2_squared: -81.4313
Epoch 3/454
8/8 [==============================] - 0s 3ms/step - loss: 5095.4131 - mae: 70.9006 - r2_squared: -78.4738
Epoch 4/454
8/8 [==============================] - 0s 3ms/step - loss: 4979.4907 - mae: 70.0767 - r2_squared: -78.2360
Epoch 5/454
8/8 [==============================] - 0s 3ms/step - loss: 4835.6313 - mae: 69.0373 - r2_squared: -72.9122
Epoch 6/454
8/8 [==============================] - 0s 2ms/step - loss: 4655.5400 - mae: 67.7172 - r2_squared: -71.4930
Epoch 7/454
8/8 [==============================] - 0s 2ms/step - loss: 4435.5083 - mae: 66.0586 - r2_squared: -69.0396
Epoch 8/454
8/8 [==============================] - 0s 3ms/step - loss: 4169.5352 - mae: 63.9975 - r2_squared: -65.6598
Epoch 9/454
8/8 [==============================]

In [ ]:
# Evaluate the final model
tuned_prediction = tuned_final_model.predict(X_test)

8/8 [==============================] - 0s 2ms/step


In [ ]:
tuned_final_mse = mean_squared_error(Y_test, tuned_prediction)
print(f"tuned_Final Mean Squared Error on the best model: {tuned_final_mse:.2f}")

tuned_Final Mean Squared Error on the best model: 33.15


In [ ]:
tuned_final_mae = mean_absolute_error(Y_test, tuned_prediction)
print(f"Final Mean Absolute Error on the best model: {tuned_final_mae:.2f}")

Final Mean Absolute Error on the best model: 4.44


In [ ]:
tuned_final_r2 = r2_score(Y_test, tuned_prediction)
print(f"Final r2_score on the best model: {tuned_final_r2:.2f}")

Final r2_score on the best model: 0.41
